In [1]:
# standard imports
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
from tqdm.auto import tqdm
import time
from datetime import timedelta
from collections import Counter, defaultdict
# import polars as pl

# import defined file paths 
from config import *

# data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

import warnings
# warnings.filterwarnings('ignore')  ## uncomment to surpress warnings

https://www.kaggle.com/code/edwardcrookenden/otto-getting-started-eda-baseline

In [3]:
# train = pd.read_parquet(OPT_TRAIN_PATH)
# test = pd.read_parquet(OPT_TEST_PATH)

In [4]:
with open(TRAIN_PATH, 'r') as f:
    print(f'{len(f.readlines()):,} lines in the training data')

12,899,779 lines in the training data


In [5]:
with open(TEST_PATH, 'r') as f:
    print(f'{len(f.readlines()):,} lines in the training data')

1,671,803 lines in the training data


In [6]:
sample_size = 200000

chunks = pd.read_json(TRAIN_PATH, lines=True, chunksize=sample_size)

for c in chunks:
    train_sample = c.set_index('session')
    break

In [7]:
train_sample.head()

,events
session,
0,"[{'aid': 1517085, 'ts': 1659304800025, 'type':..."
1,"[{'aid': 424964, 'ts': 1659304800025, 'type': ..."
2,"[{'aid': 763743, 'ts': 1659304800038, 'type': ..."
3,"[{'aid': 1425967, 'ts': 1659304800095, 'type':..."
4,"[{'aid': 613619, 'ts': 1659304800119, 'type': ..."


In [8]:
first_session = train_sample.iloc[0].item()
print(f'The first session had {len(first_session)} actions')
# Time of session
time_elapsed = first_session[-1]["ts"] - first_session[0]["ts"]
# The timestamp is in milliseconds since 00:00:00 UTC on 1 January 1970
print(f'The first session elapsed: {str(timedelta(milliseconds=time_elapsed))}')

action_counts = defaultdict(int)
for action in first_session:
    a_type = action['type']
    action_counts[a_type] = action_counts[a_type] + 1
    
print(f'The first session had this frequency of actions: {action_counts}')


The first session had 276 actions
The first session elapsed: 27 days, 13:09:43.682000
The first session had this frequency of actions: defaultdict(<class 'int'>, {'clicks': 255, 'carts': 17, 'orders': 4})


In [9]:
train_sample.iloc[0].item()

[{'aid': 1517085, 'ts': 1659304800025, 'type': 'clicks'},
 {'aid': 1563459, 'ts': 1659304904511, 'type': 'clicks'},
 {'aid': 1309446, 'ts': 1659367439426, 'type': 'clicks'},
 {'aid': 16246, 'ts': 1659367719997, 'type': 'clicks'},
 {'aid': 1781822, 'ts': 1659367871344, 'type': 'clicks'},
 {'aid': 1152674, 'ts': 1659367885796, 'type': 'clicks'},
 {'aid': 1649869, 'ts': 1659369893840, 'type': 'carts'},
 {'aid': 461689, 'ts': 1659369898050, 'type': 'carts'},
 {'aid': 305831, 'ts': 1659370027105, 'type': 'orders'},
 {'aid': 461689, 'ts': 1659370027105, 'type': 'orders'},
 {'aid': 362233, 'ts': 1659370064916, 'type': 'clicks'},
 {'aid': 1649869, 'ts': 1659370067686, 'type': 'clicks'},
 {'aid': 1649869, 'ts': 1659371003682, 'type': 'clicks'},
 {'aid': 984597, 'ts': 1659371033243, 'type': 'clicks'},
 {'aid': 1649869, 'ts': 1659371042297, 'type': 'clicks'},
 {'aid': 803544, 'ts': 1659371044075, 'type': 'clicks'},
 {'aid': 1110941, 'ts': 1659371104329, 'type': 'clicks'},
 {'aid': 1190046, 'ts': 

In [10]:
dict(Counter(action['aid'] for action in train_sample.iloc[0].item()))

{1517085: 1,
 1563459: 1,
 1309446: 1,
 16246: 1,
 1781822: 1,
 1152674: 1,
 1649869: 4,
 461689: 3,
 305831: 3,
 362233: 3,
 984597: 2,
 803544: 1,
 1110941: 1,
 1190046: 2,
 1760685: 1,
 631008: 1,
 1650637: 2,
 313546: 1,
 979517: 1,
 351157: 1,
 1062149: 1,
 1157384: 1,
 1841388: 1,
 1469630: 1,
 1110548: 3,
 1650114: 1,
 1604396: 1,
 1009750: 1,
 1800933: 1,
 495779: 2,
 394655: 1,
 789245: 3,
 366890: 1,
 361317: 1,
 1700164: 2,
 1755597: 1,
 784978: 3,
 1171505: 1,
 1521766: 2,
 1725503: 1,
 528847: 1,
 1816325: 1,
 1072782: 5,
 173702: 2,
 1407538: 1,
 1629651: 1,
 1768568: 1,
 1318324: 2,
 1840418: 1,
 1813509: 4,
 667924: 1,
 1226444: 1,
 709550: 1,
 709417: 1,
 1225559: 2,
 1048044: 1,
 1052813: 1,
 240346: 1,
 1582117: 1,
 1707783: 1,
 1624436: 3,
 1157411: 3,
 358305: 1,
 1202970: 1,
 832192: 1,
 1498443: 1,
 723931: 1,
 1436439: 4,
 1693461: 1,
 1206554: 1,
 1110741: 1,
 346352: 1,
 1802050: 1,
 154930: 1,
 964169: 6,
 823637: 1,
 1411683: 1,
 1167722: 1,
 1619737: 1,
 18

In [11]:
action_type_count_list, aid_count_list, session_duration_list, session_action_count_list = ([] for _ in range(4))
# frequency of actions of each type (click, cart, order)
total_action_type_counts = Counter()
# frequency of article ids
total_aid_counts = Counter()
for session, actions in tqdm(train_sample.itertuples(), total=len(train_sample)):
    # count the frequency of each action type 
    action_type_count = Counter(action['type'] for action in actions)
    # count the frequency of each article id
    aid_count = Counter((action['aid'] for action in actions))
    
    # add action and aid counts from this session to the total
    total_action_type_counts += action_type_count
    total_aid_counts += aid_count
    
    # record the session duration
    session_duration = actions[-1]['ts'] - actions[0]['ts']
    
    # append to lists
    action_type_count_list.append(dict(action_type_count))
    aid_count_list.append(dict(aid_count))
    session_duration_list.append(session_duration)
    session_action_count_list.append(len(actions)) # count of actions in this session is the length of the actions array

train_sample['action_type_counts'] = action_type_count_list
train_sample['aid_counts'] = aid_count_list
train_sample['duration'] = session_duration_list
train_sample['action_count'] = session_action_count_list

  0%|          | 0/200000 [00:00<?, ?it/s]